# Brainstorming for astroquery.vo needs


If NAVO develops astroquery.vo, we could use things like the following. This is a summary of what is below in more detail. 

RegTAP:  

    query_results=astroquery.vo.Registry.query( ... lots of options, this already exists in our github ...)
    heasarc_image_services=astroquery.vo.Registry.list_image_services(source='heasarc') 

SCS:

    ned_results=astroquery.vo.Cone.query(ras,decs,radius,source='ned')
    
SIA:  
    
    images_info=astroquery.vo.Image.query() 
    plt.imshow( astroquery.vo.Image.get( images_info[0] )
    astroquery.vo.Image.get( images_info[0], filename='image.fits')

SSA:

    Same as SIA basically. 

TAP(?):

    tap_services_2mass=Registry.query(keyword='2mass',service_type='table')[0]
    tap_results=Tap.query(
        source=tap_services_2mass[32],
        logic_string='CONTAINS(POINT('J2000',ra,dec),CIRCLE('J2000',9.90704,8.96507,0.001))'
        )

Going through Vandana's presentation to make a list of functions we could use, with one basic version of a Cone class defined to play with.

In [1]:
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline  
import requests, io, astropy
from IPython.display import Image, display

## For handling ordinary astropy Tables
from astropy.table import Table, vstack

## For reading FITS files
import astropy.io.fits as apfits

## There are a number of relatively unimportant warnings that 
## show up, so for now, suppress them:
import warnings
warnings.filterwarnings("ignore")

## our stuff
import sys
# Use the NASA_NAVO/astroquery
from navo_utils.cone import Cone
from navo_utils.registry import Registry

Registry queries already coded by TomD and TJ in astroquery.vo.Registry() class. So you can, for example, if you already know you want to search NED, get it's URL as follows. Unfortunately, *with the current implementation, you get two results, where the second isn't NED but has "ned" in the ivoid ("shela_combined"). Not sure what to do about that.* Could hard-wire things like "ned", "heasarc", etc. But that's not ideal. 

(I modified the logging of astroquery.vo.Registry in my version.)

In [2]:
results = Registry.query(source='ned', service_type='cone',debug=True)
print('Found {} results:'.format(len(results)))
print(results[:]['access_url'])
print(results[1]['ivoid'])
print(results.columns)

Registry:  sending query ADQL = 
          select res.waveband,res.short_name,cap.ivoid,res.res_description,
          int.access_url, res.reference_url
           from rr.capability cap
           natural join rr.resource res
           natural join rr.interface int
           where cap.cap_type='conesearch' and cap.ivoid like '%ned%'

Queried: https://vao.stsci.edu/RegTAP/TapService.aspx/sync

Found 2 results:
                                                 access_url                                                
-----------------------------------------------------------------------------------------------------------
http://ned.ipac.caltech.edu/cgi-bin/NEDobjsearch?search_type=Near+Position+Search&amp;of=xml_main&amp;&amp;
                                                     https://irsa.ipac.caltech.edu/SCS?table=shelacomb&amp;
ivo://irsa.ipac/spitzer/catalog/shela/shela_combined
<TableColumns names=('waveband','short_name','ivoid','res_description','access_url','reference_url'

The Registry.query() method takes arguments (passed to internal function _build_adql):  

    service_type   : "image", "cone", or "spectr"
    keyword        : any keyword contained in ivoid, title, or description
    waveband       : waveband string
    source         : any substring in ivoid
    order_by       : what field to order it by, but then you have to know the names, currently
                      ("waveband","short_name","ivoid","res_description", "access_url", "reference_url")
    logic_string   : any other string you want to add to the ADQL where clause, should start with " and "

The results are already in an astropy table from Tom's _astropy_table_from_votable_response(). 

**But note that the URLs are escaped and should not be by the time we get them back. How to fix?**

In [3]:
import html
print(html.unescape(results[0]['access_url']))

http://ned.ipac.caltech.edu/cgi-bin/NEDobjsearch?search_type=Near+Position+Search&of=xml_main&&


### 3. Workshop section on data discovery using NED's Cone search. 

Instead of searching NED ‘manually’, a generic cone search that you can give a list of ras, decs, and radii (or just one obviously) and optionally specify that you want ‘ned’ results or some other IVOID substring. It queries the RegTAP to find out what cone searches are available that match what you asked for. (Is there a way to get the NED URL dynamically from RegTAP without the above ambiguity? A special case if the ivoid requested is "ned"?) So the user would call:

    cone_results = astroquery.Cone.query(ras, decs, radii, [source=’some_ivoid_string_eg_ned', waveband=etc.]) 

where ras, decs, and radii can be floats, strings, or arrays of either. If a single source (i.e., ivoid), then get back a table of objects; if several matching sources (which will have different columns?), get back a list of tables, one for each matching source? Since every table will return different columns, need to return some kind of meta data result as well. Separate object or attached to each result column's meta data? Or standardize the tables as discussed below?

Note that with kwargs, you can pass through any parameters to the Registry.query() call.

So like the Registry, we need a Cone work as follows:

In [4]:
#  Single arguments:  should take floats or strings, converts floats to string for the query.
#  For now, make them all arrays until we sort the above issue
ras=[185.47873,35.323]
decs=[4.47365,6.934]
radius=[0.03]
# List arguments:  should take list of floats or strings, converts floats to strings, loops over them. 
#  Length of ras and decs should be the same, radius should be either same length or single value.
ned_results=Cone.query(ras,decs,radius,source='ned')
#  Note that the meta data isn't getting correctly merged. TO BE FIXED.
#print(ned_results[0].meta['url'])
#print(ned_results.meta["xml_raw"])

Found 2 services to query.
    Querying service http://ned.ipac.caltech.edu/cgi-bin/NEDobjsearch?search_type=Near+Position+Search&of=xml_main&&
    Got 494 results for source number 0
    Got 6 results for source number 1
    Querying service https://irsa.ipac.caltech.edu/SCS?table=shelacomb&
    (Got no results for source number 0)
    (Got no results for source number 1)


In [7]:
ned_results[0:5]

[<Table masked=True length=500>
 main_col1          main_col2           ... main_col16 main_col17
                                        ...                      
   int32             bytes30            ...   int32      int32   
 --------- ---------------------------- ... ---------- ----------
         1                  MESSIER 061 ...         11          0
         2           NGC 4303:[CGM97] H ...          0          0
         3           NGC 4303:[CGM97] A ...          0          0
         4           NGC 4303:[CGM97] F ...          0          0
         5           NGC 4303:[CGM97] I ...          0          0
         6           NGC 4303:[CGM97] D ...          0          0
         7           NGC 4303:[CGM97] J ...          0          0
         8           NGC 4303:[CGM97] E ...          0          0
         9           NGC 4303:[CGM97] G ...          0          0
       ...                          ... ...        ...        ...
       491   SSTSL2 J122147.71+042817.3 ... 

Or if you don't know what the source is but you want to do a cone search on all catalogs related to gammas:

In [8]:
gamma_results=Cone.query(ras,decs,[0.01],waveband='gamma')
print("Got results from {} different sources.".format(len(gamma_results)))

Found 404 services to query.
    Querying service http://adsabs.harvard.edu/cgi-bin/abs_connect?data_type=VOTABLE&
    Got 200 results for source number 0
    Got 200 results for source number 1
    Querying service http://vo.bsdc.icranet.org/whsp/q/cone/scs.xml?
    (Got no results for source number 0)
    (Got no results for source number 1)
    Querying service http://simbad.u-strasbg.fr/simbad/sim-cone?
    Got 8 results for source number 0
    (Got no results for source number 1)
    Querying service http://vizier.u-strasbg.fr/viz-bin/votable/-A?-out.all&-source=IX%2F20A%2Fbatse&
    (Got no results for source number 0)
    (Got no results for source number 1)
    Querying service http://vizier.u-strasbg.fr/viz-bin/votable/-A?-out.all&-source=IX%2F20A%2Fulysses&
    (Got no results for source number 0)
    (Got no results for source number 1)
    Querying service http://vizier.u-strasbg.fr/viz-bin/votable/-A?-out.all&-source=IX%2F51%2Ftable2&
    (Got no results for source number 

    (Got no results for source number 0)
    (Got no results for source number 1)
    Querying service http://vizier.u-strasbg.fr/viz-bin/votable/-A?-out.all&-source=J%2FA%2BA%2F557%2FA12%2FGRBs&
    (Got no results for source number 0)
    (Got no results for source number 1)
    Querying service http://vizier.u-strasbg.fr/viz-bin/votable/-A?-out.all&-source=J%2FA%2BA%2F558%2FA137%2Ftable5a&
    (Got no results for source number 0)
    (Got no results for source number 1)
    Querying service http://vizier.u-strasbg.fr/viz-bin/votable/-A?-out.all&-source=J%2FA%2BA%2F559%2FA9%2Ftable3&
    (Got no results for source number 0)
    (Got no results for source number 1)
    Querying service http://vizier.u-strasbg.fr/viz-bin/votable/-A?-out.all&-source=J%2FA%2BA%2F562%2FA64%2Ftable3&
    (Got no results for source number 0)
    (Got no results for source number 1)
    Querying service http://vizier.u-strasbg.fr/viz-bin/votable/-A?-out.all&-source=J%2FA%2BA%2F562%2FA70%2Flist&
    (Got no r

    Querying service http://vizier.u-strasbg.fr/viz-bin/votable/-A?-out.all&-source=J%2FApJ%2F633%2FL77%2Ftable1&
    (Got no results for source number 0)
    (Got no results for source number 1)
    Querying service http://vizier.u-strasbg.fr/viz-bin/votable/-A?-out.all&-source=J%2FApJ%2F636%2F765%2Ftable2&
    (Got no results for source number 0)
    (Got no results for source number 1)
    Querying service http://vizier.u-strasbg.fr/viz-bin/votable/-A?-out.all&-source=J%2FApJ%2F649%2FL9%2Ftable1&
    (Got no results for source number 0)
    (Got no results for source number 1)
    Querying service http://vizier.u-strasbg.fr/viz-bin/votable/-A?-out.all&-source=J%2FApJ%2F657%2F706%2Ftable2&
    (Got no results for source number 0)
    (Got no results for source number 1)
    Querying service http://vizier.u-strasbg.fr/viz-bin/votable/-A?-out.all&-source=J%2FApJ%2F681%2F113%2Ftable1&
    (Got no results for source number 0)
    (Got no results for source number 1)
    Querying service 

    (Got no results for source number 0)
    (Got no results for source number 1)
    Querying service http://vizier.u-strasbg.fr/viz-bin/votable/-A?-out.all&-source=J%2FApJ%2F754%2F121%2Ftable1&
    (Got no results for source number 0)
    (Got no results for source number 1)
    Querying service http://vizier.u-strasbg.fr/viz-bin/votable/-A?-out.all&-source=J%2FApJ%2F754%2F23%2Ftable1&
    (Got no results for source number 0)
    (Got no results for source number 1)
    Querying service http://vizier.u-strasbg.fr/viz-bin/votable/-A?-out.all&-source=J%2FApJ%2F756%2F112%2Fgrb&
    (Got no results for source number 0)
    (Got no results for source number 1)
    Querying service http://vizier.u-strasbg.fr/viz-bin/votable/-A?-out.all&-source=J%2FApJ%2F756%2F33%2FXsources&
    (Got no results for source number 0)
    (Got no results for source number 1)
    Querying service http://vizier.u-strasbg.fr/viz-bin/votable/-A?-out.all&-source=J%2FApJ%2F756%2F44%2Ftable9&
    (Got no results for 

    Querying service http://vizier.u-strasbg.fr/viz-bin/votable/-A?-out.all&-source=J%2FApJS%2F180%2F192%2Ftable2&
    (Got no results for source number 0)
    (Got no results for source number 1)
    Querying service http://vizier.u-strasbg.fr/viz-bin/votable/-A?-out.all&-source=J%2FApJS%2F183%2F46%2Ftable5&
    (Got no results for source number 0)
    (Got no results for source number 1)
    Querying service http://vizier.u-strasbg.fr/viz-bin/votable/-A?-out.all&-source=J%2FApJS%2F186%2F1%2Ftable3&
    (Got no results for source number 0)
    (Got no results for source number 1)
    Querying service http://vizier.u-strasbg.fr/viz-bin/votable/-A?-out.all&-source=J%2FApJS%2F188%2F405%2Ftable2&
    (Got no results for source number 0)
    (Got no results for source number 1)
    Querying service http://vizier.u-strasbg.fr/viz-bin/votable/-A?-out.all&-source=J%2FApJS%2F188%2F405%2Ftable7&
    (Got no results for source number 0)
    (Got no results for source number 1)
    Querying servi

    (Got no results for source number 0)
    (Got no results for source number 1)
    Querying service https://heasarc.gsfc.nasa.gov/cgi-bin/vo/cone/coneGet.pl?table=batsetrigs&
    (Got no results for source number 0)
    (Got no results for source number 1)
    Querying service https://heasarc.gsfc.nasa.gov/cgi-bin/vo/cone/coneGet.pl?table=cbatpicagn&
    (Got no results for source number 0)
    (Got no results for source number 1)
    Querying service https://heasarc.gsfc.nasa.gov/cgi-bin/vo/cone/coneGet.pl?table=cgrotl&
    (Got no results for source number 0)
    (Got no results for source number 1)
    Querying service https://heasarc.gsfc.nasa.gov/cgi-bin/vo/cone/coneGet.pl?table=comptel&
    (Got no results for source number 0)
    (Got no results for source number 1)
    Querying service https://heasarc.gsfc.nasa.gov/cgi-bin/vo/cone/coneGet.pl?table=cosbmaps&
    (Got no results for source number 0)
    (Got no results for source number 1)
    Querying service https://heasarc.

    (Got no results for source number 0)
    (Got no results for source number 1)
    Querying service https://heasarc.gsfc.nasa.gov/cgi-bin/vo/cone/coneGet.pl?table=sas3ylog&
    (Got no results for source number 0)
    (Got no results for source number 1)
    Querying service https://heasarc.gsfc.nasa.gov/cgi-bin/vo/cone/coneGet.pl?table=saxgrbmgrb&
    (Got no results for source number 0)
    (Got no results for source number 1)
    Querying service https://heasarc.gsfc.nasa.gov/cgi-bin/vo/cone/coneGet.pl?table=sterngrb&
    (Got no results for source number 0)
    (Got no results for source number 1)
    Querying service https://heasarc.gsfc.nasa.gov/cgi-bin/vo/cone/coneGet.pl?table=swiftbalog&
    Got 42 results for source number 0
    (Got no results for source number 1)
    Querying service https://heasarc.gsfc.nasa.gov/cgi-bin/vo/cone/coneGet.pl?table=swiftgrb&
    (Got no results for source number 0)
    (Got no results for source number 1)
    Querying service https://heasarc

In [9]:
print(gamma_results[2])

target_id    obsid        ra    ... operation_mode pointing_mode Search_Offset
                         deg    ...                                           
--------- ----------- --------- ... -------------- ------------- -------------
    33001 00033001052 185.47122 ...   Rates_Arr_1s      POINTING        0.4684
    33001 00033001052 185.47122 ...   Rates_Arr_1s      POINTING        0.4684
    33001 00033001052 185.47122 ...          Event SLEW_POINTING        0.4684
    34597 00034597008 185.47122 ...         Survey      POINTING        0.4684
    34597 00034597008 185.47122 ...         Survey      POINTING        0.4684
   704772 00704772000 185.47122 ...          Event SLEW_POINTING        0.4684
   704773 00704773000 185.47122 ...          Event SLEW_POINTING        0.4684
   704774 00704774000 185.47122 ...          Event SLEW_POINTING        0.4684
   704775 00704775000 185.47122 ...          Event SLEW_POINTING        0.4684
   704776 00704776000 185.47122 ...          Event S

*Notes: the _astropy_table_from_votable_response() should then be generic, not just in Registry class*

But these come with different columns:

In [10]:
print(gamma_results[0].columns)
print(gamma_results[6].columns)


<TableColumns names=('MAIN','ra_j2000','dec_j2000','bibcode','title','creator','source','date','url')>
<TableColumns names=('main_col1','main_col2','main_col3','main_col4','main_col5','main_col6','main_col7','main_col8','main_col9','main_col10','main_col11','main_col12','main_col13','main_col14','main_col15','main_col16','main_col17')>


There's only a UCD in some columns, and it depends on the service. 

**So to merge into standard tables, perhaps go through looking for UCDs or UTYPEs and renaming any columns with them with that standard name. Then do the merge.**  If you use the default 'outer' join, you'll end up with lots of columns where rows from different services use differnet columns and the others remain empty. Give the user the option to do an 'inner' join, and you'll end up only with columns that are common to all results, probably only the ones with the UCDs. 


Started to define a function for the second part of this cell in the workshop notebook that got the pass bands from NED. This is very NED-specific. Any way to generalize?

    ned_info = astroquery.get_ned_info( ra, dec, radii )

calls the cone search and passes the ACREF for each match to NED again to get the info. But ACREF isn't a required value returned by a cone search. All that's required is the ID, RA, and DEC. So I don't think this can be generalized.


## 4. 

    sia_services = astroquery.vo.list_sia_services( [source=’ivoid_string_eg_heasarc'] , [keyword=‘allwise’], [waveband=‘whatever’] ... ) 

This one can easily be generalized so you can get images from any service (or a chosen one) using the same options as the Registry.query(), i.e., waveband, keyword, etc. 

It returns a table of information, including the ‘access_url’ that you can then plug into another generic function




In [11]:
def list_image_services(**kwargs):
    return astroquery.vo.Registry(service_type="image",**kwargs)
def list_spectra_serices(**kwargs):
    return astroquery.vo.Registry(service_type="spectra",**kwargs)
def list_cone_serices(**kwargs):
    return astroquery.vo.Registry(service_type="cone",**kwargs)
#  This one isn't in Registry yet, but presumably can be added.
def list_tap_services(**kwargs):
    return astroquery.vo.Registry(servic_type="tap",**kwargs)

## 5 SIA

Then pick one of the listed services (say number 20, after you looked at the descriptions) and query it to get the URL to an image at a given coordinate.

    image_info = astroquery.vo.Image.query( access_url=sia_services[20][‘access_url’], pos=‘185.47873,4.47365', size=‘0’]

or perhaps you don't know which service is quite what you want, so get info for all of them:

    images_info = astroquery.vo.Image.query( access_url=sia_services[:][‘access_url’], pos=‘185.47873,4.47365', size=‘0’, naxis=‘300,300’]

to get a table list of all images in a list of services that contain that point. 

The standardize() function would be a version of _astropy_table_from_votable_response.

Or, like with the Cone search above, the user doesn't give a service but just asks for information on images matching whatever criteria:

    heasarc_images_info = astroquery.vo.Image.query( 
        pos=‘185.47873,4.47365', size=‘0’, naxis=‘300,300’, service='heasarc')
    uv_images_info = astroquery.vo.Image.query( 
        pos=‘185.47873,4.47365', size=‘0’, naxis=‘300,300’, waveband='uv')
    2mass_images_info = astroquery.vo.Image.query( 
        pos=‘185.47873,4.47365', size=‘0’, naxis=‘300,300’, keyword='2mass')


## 7-8. Retrieving images

You can look at the images_info and pick one to download:

    astroquery.vo.Image.get( images_info[6], filename='my_file.fits')
    
or get the image data to hand to the plotter:

    image=astroquery.vo.Image.get( images_info[6] )
    plt.imshow( image,  cmap='gray', origin='lower',vmax=0.02 )
    
where it downloads it to filename if specified, or to a temporary filename if not and then reads it in and returns the image data if not.

Or hand a list and return a list of images:

    images=astroquery.vo.Image.get(images_info )
    plt.imshow( images[0],  cmap='gray', origin='lower',vmax=0.02 )

In [12]:
class ImageClass(BaseQuery):
    
    def query(self, **kwargs):
        """Get information on what images are available"""
        services=Registry.query(service_type='image',**kwargs)
        # Like the Cone class above, collect results...
        return 
    
    def get(self, image_url , filename=''):
        """Returns the data that can be handed to plt.imshow() from a URL
    
        For now, input URL. But could just get a list of URLs or a 
        list of tables that have an 'access_url' column.
        """
        if filename is '':
            filename='tmp.fits'
        self._download( image_url, filename=filename)
        if filename == 'tmp.fits':
            hdus=astropy.io.fits.open('tmp.fits')
            # Which extension? TBD
            return hdus[0].data
        else:
            return

    def _download(self, url, filename=''):
        # simple wrapper of urllib
        return
    
class SpectralClass(ImageClass):

SyntaxError: unexpected EOF while parsing (<ipython-input-12-e6e09ea3ab86>, line 29)

## 10. 

This is now easy:

    chandra_results=Cone.query(ras,decs,radius,keyword='chandra')
    
(But that's a lot of stuff. Combining with source='heasarc' currently not working, **TBFixed**.)

# 11. TAP

This currently doesn't work but should be perfectly doable:

    tap_services_2mass=Registry.query(keyword='2mass',service_type='table')
    
Look through the results, find the one you want, then assuming you know how to construct ADQL logic and you know the names of the columns in the catalog you're searching:

    tap_results=Tap.query(
        source=tap_services_2mass[32],
        logic_string='CONTAINS(POINT('J2000',ra,dec),CIRCLE('J2000',9.90704,8.96507,0.001))'
        )

is the equivalent to a cone search, but you could do whatever you wanted. If you didn't know what TAP service you wanted, you probably couldn't do this (as above for images, where you can get image information from all services in the registry). The reason is that the TAP query would depend on the column names, and they are not common.

On the other hand, since people have to know how to use ADQL and know the columns of the catalog they're interested in, it's not clear we can add much value with a wrapper.